In [1]:
from sage.all import *
import base64
import binascii
import struct
import hashlib
from sage.crypto import util
from itertools import combinations
import numpy
import struct
import socket

# Hashed ELGAMAL

In [2]:
Q1_good = ('Do8EMrud7y7ryo3mFVupDgIdkzfQOsXm6nVNOGcu1rawrmUAIWVC4l9oUjpc8H58', 'Y2FydHdoZWVsZWQgcGhpbG9zb3BoaWNhbGx5IGVhZ2VybHkgYXBwcm94aW1hdGluZyBkZW1vZ3JhcGhlciBtZXRhYm9saXNtcyBleGVjdXRvcnMgYmlvY2hlbWlzdHMgZXBpdG9taXplcyBleHRpcnBhdGlvbiBvcnRob3BhZWRpc3RzIGRlcGFydG1lbnRhbGl6ZSBqZXNzYW1pbmUgYmxhc3BoZW1pbmcgcmVjb25uYWlzc2FuY2VzIGZhbHNldHRvcyBjYXJlbGVzc25lc3MgZW5jbG9zdXJlIGlzIGV1cGhlbWlzdGljYWxseS4=', '2AYF23lYmI6GUWd3n7cUJW+uKiqGKUcXuuNa0yiZbt2yg1sZFgGEZFFOji6UUUvk64T/UcTFelefqVSeUsY5rQIJqs7/O5T57xddxFzDIxlrljG5UQuy2r1kND0ygL9oeLEG5HFWvAwqKepReEwdMzlzWmgDwx/23sYlxi9Rhh0ARWa1hhw+n4zWwAUkvXe0x1HedjTDUEHxtLgSqK5Rqfbn5GfQBYL5g+P0Uw8eQqwsC612efF70AJzGSDWvFABC2Gov9sBWL7HrUjDGt9EEWhOgtHc9MjofKuJ29J0asjpWIYv7ldPTEW2ASGVyL5cvCbCsWWE6HJ63z+o1M4i1q+ShwTAX3FRncD6b4lO7pxRlgQjg6l+9+SRaFgD4bkP')
Q1_bad = ('CBIjW/ObQkQKUL11nOETtbCnSoLqCNoQIXu69PdmVnxQxMfGMAfBhLQZtQUPh855', 'bWlzdW5kZXJzdGFuZGluZ3MgZWVyaWVzdCBiZWF1dGVvdXNseSB0cmFuc3Zlc3Rpc20gcmVtaW5pc2NlbmNlcyBjb25mcm9udGF0aW9uIHVuY2VyZW1vbmlvdXNseSBjb21tYW5kbWVudHMgcHJvcGhldGVzc2VzIGFzc2ltaWxhdGVzIGluY2VuZGlhcmllcyBhY2FudGh1c2VzIGhlbWF0b2xvZ2lzdCBtZWRhbGxpb25zIGFjcXVpZXNjZXMgcmVwb3NzZXNzZXMgYm9tYmFyZG1lbnRzIG92ZXJzdGF0ZW1lbnRzIGFiYmVzc2VzIHBhcmVudGhldGljYWxseS4=', 'X1eiBJgpCnyGsmO4qYarnP4xBr9lBhpDQaN4hKRGcgdz1vbPy0P3YQTJRqsdVpGqZ0kqzcfWPsVt9srK3YIBtqIzDYQnLy2f3JvYF6ImIGuH4+M/d5xN9hRE+fWCNDBdBna/di4Tz3zr9DEfU5cIxRnT3lj3JkJVzEvhyhjHQGnZ701Xe0UHQwhy5CoTd6l2Xy4ywInrLcvdhoOv+BGaCWn4YgOS2Ch7nqgkLkX3ylfQ1lH5G6G1YCD/fnzxSsuvMUxRKBRDF2JCikFB1nsSxRqFBxQVqVP4UJs94aZimY2SQnnQvXB0uCih45o4NsZM8bgkEtIqX7PpKaGPbNpPWwYcKg28beq4fgGhR5qBw8zkUseFcCurphgDukPvqJhnPnz+cf2wBBFdHdHqQfe5P2z/')
Q1_list = [('Urbu1qU3iYnI96MF+vqCbkM3uH1AjVbhuCxMaNM3xQ0IpeGFJR7J72ttlXIgcFZZ', 'YWxpbWVudGFyeSBtYXRoZW1hdGljaWFucyB0aHVuZGVyYm9sdHMgYXBwbGljYXRvcnMgbWlzZGVtZWFub3IgYWJydXB0ZXN0IGJlYWNoY29tYmVyIGNvbWZvcnRhYmx5IG1pc2NhbGN1bGF0aW5nIHVudGltZWxpZXIgb3B0b21ldHJ5IG15c3RlcmlvdXNseSBkaWZmdXNlbmVzcyBjaGFyYWN0ZXJpc3RpY2FsbHkgZ2VudGxld29tZW4gYWJzdHJhY3RuZXNzZXMgc3VwZXJpbnRlbmRlbnRzIGFjYW50aGkgcG9ydG1hbnRlYXVzIHRyYW5zZ3Jlc3Nlcy4=', 'pxgidFaV8URd05/toZrt/Z40dGXfRE/PSvlT6y/RFtyobnbymLImTiukYwQYufQWIxvPOZ82/VvUO7CEzcvnwj8M8zJIRrinmg/ARkLZYnaOlvuuCyZdNhGpy+oCYk58CTytTiCZLHOf8XE8qgQvRi8X4vO4FmDYvxxgXYEzTK4p7QWNEsQEt8tdK0S0SrsuFk77ZVWmeQ7K7Mmr4R3nbjibzwrBzKlFu8A/i3KE/SXMPF6oU07qfdi8ptJIbj15/QP2aM86EcCVwhbaGDdtuWpia6ZYx2eEHFyvrTvkLWh3tERHj2bs4IMNXlXUdLn2Th6xTXkGTsNbinXciew2kZJV2hL7LMaAy03KHL9k5jSU3xLE9ZgE3EUdf8bSMU9EV6P7FVqCxMdGgvgySo3/'), ('HWyAGLyKK1A1MW6wXfnNXertmv5anshScLIIeeRjiQG4gVgNPfpCNICNHz2qz71P', 'aGFsbHVjaW5vZ2VuaWMgZW50cnVzdGVkIGNhc3NldHRlcyBhcHByb3hpbWF0ZWx5IHN1YnNjcmlwdGlvbiB0ZXJtaW5hdG9ycyBjb3JyZWxhdGl2ZXMgaW5kZXBlbmRlbnRseSBhc3NlbWJsZXJzIG1lc21lcml6aW5nIHNldmVudGVlbnRocyBlZmZlbWluYXRlIGJsYWNrc21pdGhzIG1hZ25ldG9zcGhlcmUgaHlwZXJzZW5zaXRpdml0aWVzIGJpYmxpb2dyYXBoaWVzIHVuZGlzY292ZXJlZCBleHBhbnNpb25pc3QgYmVhY2hoZWFkcyBjb21taXNzYXJpYXQu', 'upol/OGg7hM4JDho41cx3zcKVA4oZzsqky1cmyacGzTeS8Hkdyz8dIoNJ6oKTmYmOjIPqo7FHRQT5pB1n2xn1WV0ThCmSGNMUy+PxQ6Hg96Gk0EGqmT/5KGQvxEKwtnsXlloCkETqliio42zSRcwzOIWN/bnn/eRLVBrK1oBLgI18ZpU9keDEdn+ofZfiziqf0aPZyMiAb3Yr51XFTEoT+V352diGrJjkG13rpOXsfvIhWM2PIUWTXRxodPmZ7A4I4v1D19n7rgjAU+S4lMFmXqOKO5redmYlndBHdU4/Ry84Xgc7c3EHD5TGcV0xIgdmG/zwZXAbc/MMG9f2rxuOULPSJjsQMseoruZUSAPtP+nZti8qkqVt8S5Pdawo/3ZBhS9kwmfWvuwS6PbWGOVDJcU/w=='), ('ZfeuPCOxzTMS4S8+LAMUftcDcCk+AVXVZSKEbFaRh7UeoDGa1ibY+fVdfQsljV6u', 'ZXBpdG9taXppbmcgZGVwYXJ0bWVudGFsIGVxdWlsYXRlcmFscyBlYXJwaG9uZSBndWFyYW50ZWVpbmcgYXNzZW1ibGluZyB0cmFuc2xpdGVyYXRlZCBvdmVydGhyb3dpbmcgaW1wZXJzb25hdGluZyBzbm9ya2VsbGluZyBhbnRlbG9wZSBmYWxzaWZpY2F0aW9uIG5hdGlvbmFsaXphdGlvbnMgdGhlb3JpemVkIGJsb2NrYnVzdGVyIGVoIHNlbWl0cmFpbGVycyBhY2N1c2luZyBlZ2diZWF0ZXJzIGJvdWlsbGFiYWlzc2VzLg==', 'zY6/H/kBj393DOHW29JAFLTXrxycWe4WmstfnOW6I8UdKL3Q9yQTDjfMP928jvuQ8mVUw/ISUPhDkeyiJH7mOND0WzVVmzsR8hytUetFIBhKLvCaC9cMGguhFqI/O9Gyr80QKcd+JPpmFcxQM5GqDpgBJuoMyq9xHaV0fhI8U8HBjmAEOiwbjq8L5igb8yU04rgTVsLn7+NUdhGRcWcSqjAXoL34DPbZnkukduuXCrjIqK48rcKfIAaPSAHUEEsTZux/3hkf0dnCDcMTR2DRcAG3vl4GSmXrIWebPSWsgpk6JnWWOIVdPygmwA4yvCKboquqaRs7G2O7+5MLTMvoJ7iPH+rXNApuTsXnUp275Dt2Goi1skPhNsubIjkvbP8='), ('EghUhKdwSfrO96b8okI1Wb5xe+u7ZsNQW777IoclDWv2fFNjxriO6W82ZeCb0W98', 'ZW5jaGFudG1lbnRzIGV4dGVudWF0aW9uIHBlcnBsZXhpdGllcyBjb250YW1pbmF0aW9uIHJhcHByb2NoZW1lbnRzIGFjY3JlZGl0aW5nIHVuaW50ZXJlc3RlZCBwYXJlbnRoZXNpemluZyBkaXNhc3NvY2lhdGUgYWRtaW5pc3RlciBkaXNjb21maXR1cmUgaGFuZHdyaXR0ZW4gcXVhcnRlcm1hc3RlcnMgYmVhdXRlb3VzbHkgY291bnRlcnNpbmtpbmcgZXh0aXJwYXRlcyBvdmVyYW54aW91cyBhZmZpbGlhdGlvbnMgaW5mcmFzdHJ1Y3R1cmVzIGV4cGVjdG9yYW50Lg==', '89tvLTKcwCj+DNZRlMoiiQ68wUkLLoXsEr3Kv82SqUXYFWZlI3Z3H+KoBp75aOTpSFloWtDo/xu86l/i67atKcsmdbU60UZQXwXdCB6o3GClUa3AgN1Ylj6x5r36teFmpotsdmdSOwIhCsH+2eTD9/UWMB1wfgHEYwcZfXigypfu2yG+VNZyil4Okzxn+eDt00rb6IhFaSeNLowfuBrf/j3FtLi/Tuf+BvCG/YZEwh+bRlrB8WDaDWRWTGGCTaMkMhvAlogmlA+XRgleKa+UvYJWKe6LwgZD6rmupAjT1541rxpJ/bMIIN0AatDNXi/aLKY685o73Pezw314C42nPSh5zHsicECyBUx+hodWZwnw/3qHyM7333R8G1+FH/OnTOhc3hbtfO4XV9IchlQFN5401BBDXA8='), ('Iky5P28bVFSMYpVPD/yvTmViXoY5wn0QFqSFWJ/wYNPcwFTcKFIzd4Cn1VE4Iobh', 'c2VkdWN0aXZlbHkgc2xhdWdodGVyaG91c2VzIGFjdGlvbiBwc3ljaG9sb2dpY2FsbHkgZGlzc2ltdWxhdGluZyBkaXNzZW1ibGVkIGNvc21vbG9naXN0cyB1bmNoYXJhY3RlcmlzdGljYWxseSBzdHJlYW1saW5pbmcgZWxlY3Ryb2VuY2VwaGFsb2dyYW1zIGFubm90YXRpb24gY29udGFtaW5hdGlvbiBzcHJlYWRzaGVldHMgc3RyaXB0ZWFzZXMgZGVjZWl0ZnVsbmVzcyBjaGxvcm9mbHVvcm9jYXJib25zIG92ZXJ0dXJuaW5nIGJhbmRvbGVlcnMgc2hvcnRjaGFuZ2luZyBjb3VudGVuYW5jaW5nLg==', 'WiZye3T7+W+Occ531XXmukmCMqQqGqVQUBvGvLf1p5nStoWoNAz1z3xwOFadWHCWqIU4q06A0+AoE0UxozZvvVK2p88+rY/Mgr1F4rrVQWNhwJx567AwHw9rlmox8r7KL+tpSTYpYVA8Ok7hWRA31H5zVFMJrf0jBoSTzGzSRzQ++hT/EPpGB8i4jPAVxM3z3aTrqe98jHzuQt8dizZe918z9I5I01VyWcYGjta7j+ggW3Y0bGW84efv31GMvCN5beJMt83PHE57Yx6Z+pRYHPDeeyGWeYS3sYXUlaSviWIe64FuDh9EER1EFa/xErl/NUvpcJBk2PcvO2iZzRTqixPcvHkT4y3sQuCP9VbBM60KAYSklRZHeVmcuQ8eQ0kJZc90z+TPHItXfZQy+MI9wdaQ56adpqX3Eajhp6O24XkmoeFAQfHpYv8=')]
Q1_pk = 'cpdqVVakZhL//AD3n6fWvEaU1PMnsBbMjGtxLKPPAkMjQThiTZyebtccm7qR84ofDw=='
Q1_m = 'Y2VudHJpZnVnZXMgdGVsZWNvbW11bmljYXRpb24gYmlydGhzdG9uZXMgYXBwcm94aW1hdGVzIGFicmFzaXZlbmVzcyBhbHBoYWJldGl6ZWQgdGVldG90YWxsZXIgYmxvY2toZWFkcyBiZWNhbG1pbmcgZWNvbG9naWNhbCBlbHNld2hlcmUgYnVzaW5lc3NsaWtlIGNvY2tzY29tYnMgZ3JlZW5ob3VzZXMgdW5kZXJlc3RpbWF0ZSBtYXJtb3NldHMgc29saWNpdG91c2x5IHRyYW5zY2VuZGVudGFsaXNtIGV4dHJhdmVydHMgaGFsbHVjaW5vZ2VuaWNzLg=='
Q1_x = 7865153063309199325895146699540685310850067635968336867528075485158673984696354316621989782751410324009003258450207

In [3]:
def base2byte(element):
    return base64.b64decode(element)

In [4]:
def byte2base(element):
    return base64.b64encode(element)

In [5]:
def byte2hex(element):
    return binascii.hexlify(element)

In [6]:
def hex2byte(element):
    if len(element)%2 != 0 :
        element = '0' + element
    return binascii.unhexlify(element)

In [7]:
def byte2bin(element):
    return str(util.ascii_to_bin(element))

In [8]:
def bin2byte(element):
    return str(util.bin_to_ascii(element))

In [9]:
def hex2bin(element):
    return byte2bin(hex2byte(element))

In [10]:
def bin2hex(element):
    return byte2hex(bin2byte(element))

In [11]:
def hex2int(element):
    temp = int(element,16)
    return temp

In [12]:
def int2hex(element):
    temp = hex(element)
    if temp[-1] == 'L':
        temp = temp[:-1]
    if temp[0:2] == '0x':
        return temp[2:]
    else:
        return temp

In [13]:
def int2bin(element):
    return bin(element)[2:]

In [14]:
def bin2int(element):
    return int(element,2)

In [15]:
# create elliptic curve out of brain pool params

p = "8CB91E82A3386D280F5D6F7E50E641DF152F7109ED5456B412B1DA197FB71123ACD3A729901D1A71874700133107EC53"

A = "7BC382C63D8C150C3C72080ACE05AFA0C2BEA28E4FB22787139165EFBA91F90F8AA5814A503AD4EB04A8C7DD22CE2826"

B = "04A8C7DD22CE28268B39B55416F0447C2FB77DE107DCD2A62E880EA53EEB62D57CB4390295DBC9943AB78696FA504C11"

x = "1D1C64F068CF45FFA2A63A81B7C13F6B8847A3E77EF14FE3DB7FCAFE0CBD10E8E826E03436D646AAEF87B2E247D4AF1E"

y = "8ABE1D7520F9C2A45CB1EB8E95CFD55262B70B29FEEC5864E19C054FF99129280E4646217791811142820341263C5315"

q = "8CB91E82A3386D280F5D6F7E50E641DF152F7109ED5456B31F166E6CAC0425A7CF3AB6AF6B7FC3103B883202E9046565"

h = "1"

# reload params as integers
p = hex2int(p)
A = hex2int(A)
B = hex2int(B)
x = hex2int(x)
y = hex2int(y)
q = hex2int(q)
h = hex2int(h)

# sanity checks:

if not is_prime(p):
    print "p is not prime"

if x >= p or y >= p:
    print "tuple is not in the field"

EC = EllipticCurve(GF(p), [A,B])
G = EC(x,y)

if G == EC(0) or (q * G) != EC(0):
    print("G aint any good")

if len(bin(p)) % 8 != 2:
    print("p_len not div by 8")

In [16]:
lambda_ = len(int2bin(q)) # lambda is the bit length of p

In [17]:
def encode_field_element(elem):
    temp = int2hex(elem)
    num_zeros = lambda_/32 # from bit to hex divide by 32
    temp = '0' * (num_zeros - len(temp)) + temp  # pad with zeros
    return hex2byte(temp)

In [18]:
def decode_field_element(elem):
    
    temp = Integer(bin2int(byte2bin(elem)))
    try: # check if point has y coordinate
        EC.lift_x(temp)
        return temp
    except:
        print('x doesnt belong to curve')

In [19]:
def encode_ring_element(elem):
    temp = int2hex(elem)
    num_zeros = lambda_/32
    temp = '0' * (num_zeros - len(temp)) + temp
    return hex2byte(temp)

In [20]:
def decode_ring_element(elem):
    
    temp = Integer(bin2int(byte2bin(elem)))
    if temp < q: # check if element is in Zq
        return temp
    else:
        raise Exception('Element doesnt belong to ring')

In [21]:
def encode_point(point):
    x_coor = int(point[0])
    y_coor = int(point[1])
    x_byte = encode_field_element(x_coor)
    if is_odd(y_coor):
        x_byte = x_byte + hex2byte(int2hex(15))
    else:
        x_byte = x_byte + hex2byte(int2hex(255))
    return x_byte

In [22]:
def decode_point(point):
    
    zero_byte = hex2byte('0' + int2hex(0))
    is_infinity = True
    for i in range(0,len(point)):
        if point[i] != zero_byte:
            is_infinity = False
            break
    if is_infinity:
        return EC(0,0)
    else:
        last_byte = point[-1]
        if ((last_byte)!= hex2byte(int2hex(15))) and ((last_byte)!= hex2byte(int2hex(255))):
            raise Exception('invalid last encoding , byte = {}'.format(last_byte))
        else:
            x_coor_bytes = point[:-1]
            x_coor_int =  decode_field_element(x_coor_bytes)
            elliptic_points = EC.lift_x(x_coor_int,all=True)
            
            # select point based on parity
            point_zero = elliptic_points[0].xy()
            point_zero_y = int(point_zero[1])
            if last_byte == hex2byte(int2hex(15)):
                if is_odd(point_zero_y):
                    return elliptic_points[0]
                else:
                    return elliptic_points[1]
            else:
                if is_even(point_zero_y):
                    return elliptic_points[0]
                else:
                    return elliptic_points[1]
            
            
            return elliptic_point

In [23]:
def encode_into_domain(x):
    
    x_hex = int2hex(x)
    x_hex = '0'* (8 - len(x_hex)) + x_hex # pad to get 4 bytes
    if len(x_hex)>8:
        x_hex = x_hex[-8:] # return last 8 bytes
    x_bytes = hex2byte(x_hex)
    return x_bytes[::-1] # reverse bytes

In [24]:
def Xor(str1,str2,b):
    y = int(str1,2) ^ int(str2,2)
    return STRB(y,b)

In [25]:
def STRB(input_int,b):
    int_to_bin = bin(input_int)[2:]
    num_zeros = b - len(int_to_bin)
    int_to_bin = ('0'* num_zeros) + int_to_bin
    return int_to_bin

In [26]:
def hashstream(input_seed,msg):
    
    length = -1
    msg_length_bits = len(msg) * 8
    if msg_length_bits> (2**40):
        length = (2**40 - 1)/256
    else:
        length = int(msg_length_bits/256) + 1 # number of blocks needed
        
    blocks = ''
    for i in range(length):
        i_four = encode_into_domain(i)
        hash_block = hashlib.sha256(i_four + input_seed).digest()
        blocks = blocks + hash_block
     
    msg_bits = byte2bin(msg)
    hash_bits = byte2bin(blocks)
    
    if len(hash_bits) > len(msg_bits):
        hash_bits = hash_bits[0:len(msg_bits)]
    else:
        msg_bits = msg_bits[0:len(hash_bits)]
    
    
    
    result = Xor(msg_bits,hash_bits,len(msg_bits))
    
    return result

In [27]:
def encryption(public_key,msg):
    
    pk_byte = base2byte(public_key)
    pk_point = decode_point(pk_byte)
    x = Q1_x
    X = x * G
    K = x * pk_point
    
    X_en = encode_point(X.xy())
    K_en = encode_point(K.xy())
    l = encode_into_domain(len(msg) * 8)
        
    c = bin2byte(hashstream(l+K_en,msg))
    ct = c + X_en
    ct = base64.b64encode(ct)
    return ct

In [28]:
def decryption(secret_key,cipher_text):
    
    sk_bytes = base2byte(secret_key)
    sk = decode_ring_element(sk_bytes)
    cipher = base2byte(cipher_text)
    cipher_len = len(cipher) * 8
    cipher_len = cipher_len - lambda_ - 8
    if not ((cipher_len < (2**32)) and (cipher_len>0)):
        return "Nothing"
    else:
        x_enc_length = (lambda_ + 8)/8
        c = cipher[0:(len(cipher) - x_enc_length)]
        X_en = cipher[-x_enc_length:]
        l = encode_into_domain(len(c) * 8)
        X = decode_point(X_en)
        K = sk * X
        K_en = encode_point(K.xy())
        msg = bin2byte(hashstream(l+K_en,c))
        msg = base64.b64encode(msg)
        return msg

In [29]:
try:
    print(decryption(Q1_bad[0],Q1_bad[2]) == Q1_bad[1])
except:
    print("Algorithm failed")

x doesnt belong to curve
Algorithm failed


In [30]:
result = []
for (sk,m,ct) in Q1_list:
    try:
        msg = decryption(sk,ct)
        if msg == m:
            result.append(1)
        else:
            result.append(0)
    except Exception, e:
        print("Aborted")
        result.append(0)
result

[1, 1, 0, 1, 1]

In [31]:
m_bytes = base64.b64decode(Q1_m)
encryption(Q1_pk,m_bytes)

'DXc05PTYdXTN4cEtbuEWOhHqF2boMaqyjodLRvpSm53gLLl0ZbqSEqpakneFEDCyaKPbRy5NDfqvCh5bePjFmR1A+F6IzpD6lzOj2B2j05KhWo3RnyR2aS7ff5EDy0iWwVWId6xIv3pYn48hrlaRv1fOJO7Jr5AeJe5MrKKNTLucZnMALpKLriiTVR5s1CL27ZMgJ0Qr2PWdPXPqnJ6oePz+2IrEMERalotB8M67SLzR7Ud7x+AXQG/suPfnOD8zIJZpEnNmD9ZMzYt28Y64zFdJnWdr22/MRUc8zqIdWvIH8o2JrZCxcaUE1R3G5ayz4al/KYOPJi8BKf7ynl90N4ecv1n5qkQCcH5nz5hGtVf/C8sqQB7QkyK6wRN+LwopwUFBIbk5AfRoiu2QOw8='

# Modes of operation with counter

__NOTE__ : Please run this using sage kernel

In [2]:
IV2="yIOZSa1VYnmaEqqFjZE3/w=="
C2="5f3nmPmJGqDuQQ8IhMTKOtB8dC4B83lczYBvLR/L82FKFqq66gG9rvExSK+wrewMPN5gBJSdru5VIIGkLHYzOiKwlBbVSkHBPsCFq5p1O5PN89QA1MC5+gtDq5HgTnb6eFD+pchZS3GAbifpYrzgOWDnDvsXEFG9foTtBymldBh6hgIqGaSFDsyg4PJHuCa8560oWK301ArOT5no/u2T5TF31CHmLH0h/pEjWZlwo2KmheVJvjChbOd/GxRtGpbmnHbJ4mofCBOA86kOzwiNoxgFhz3tjzYRtt09YcCnQyQ="

In [3]:
import socket
import base64

def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(message + "\n")
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    s.close()
    return response

In [4]:
def query_server_Q2(s):
    b64 = base64.b64encode(s)
    res = connect_server("lasecpc25.epfl.ch", 6666, "281141 {}\n".format(b64))
    return base64.b64decode(res)

def BigEnd(N):
    """Input: int, output: hex"""
    N_hex = binascii.hexlify(struct.pack('>Q', N))
    return binascii.unhexlify(N_hex)

def LitEnd(N):
    """Input: int, output: hex"""
    N_hex = binascii.hexlify(struct.pack('<Q', N))
    return binascii.unhexlify(N_hex)

def XOR(a, b):
    """Bitwise XOR between binary a and b"""
    y = int(a, 2) ^^ int(b, 2)
    return bin(y)[2:].zfill(len(a))

def add(b, n):
    y = int(b, 2) + n
    return bin(y)[2:].zfill(len(b))

def replace_at(s, idx, v):
    l = list(s)
    l[idx] = v
    return "".join(l)

def XOR_byte(a, b):
    return bin_to_ascii(XOR(ascii_to_bin(a), ascii_to_bin(b)))

def bin_to_ascii(x):
    return util.bin_to_ascii(x)

def ascii_to_bin(x):
    return str(util.ascii_to_bin(x))

def int_to_bin(x, bit_length):
    temp = "{:0b}".format(x)
    temp =  temp.zfill(bit_length)
    return temp

In [5]:
base64.b64encode("Red Fox!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00")

'UmVkIEZveCEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA='

In [6]:
int((binascii.hexlify(base64.b64decode(IV2))),16)
binascii.hexlify(BigEnd(81985529216486895)), binascii.hexlify(LitEnd(81985529216486895))

('0123456789abcdef', 'efcdab8967452301')

In [16]:
### First, find out the last 2 bits of N

M0 = BigEnd(0) + LitEnd(0)
M1 = BigEnd(1) + LitEnd(1)


out1 = query_server_Q2(M1)
out2 = query_server_Q2(M1)

# If it is True, it means that the two queries have used respectively CBC and CFB correctly.
XOR_byte(out2, out1) == M1

False

In [18]:
XOR_byte(out2, out1)

'7\x9c\x1e\xa6\x83\xba[}\x95\xdb\xc5\x00\x88\x13j\xfe'

In [8]:

### Now we know that the current N value is 2 mod 4 (since 2 queries before, it was = 0 (mod 4)).

# We do another 2 queries, so that N = 0 (mod 4) again
query_server_Q2(M0); query_server_Q2(M0);

In [9]:
### We know the last 2 bits, recover the rest (except the most significant one)
N = int_to_bin(0,bit_length=64)

In [10]:
N

'0000000000000000000000000000000000000000000000000000000000000000'

In [11]:
for k in xrange(2, 63):
    # Every time, to recover the k_th last bit, we assume it is 0, and compare outputs of CBC and OFB
    # During CTR mode, we inject a message of length (k-2) blocks
    out1 = query_server_Q2(BigEnd(2^k + 3) + LitEnd(2^k + 3))
    query_server_Q2(M0)
    query_server_Q2((k-1)*M0)
    out2 = query_server_Q2(M0)
    N = replace_at(N, -(k+1), '0' if out1 == out2 else '1')
    N = add(N, 2^k + 4)

In [12]:
N

'0111111111111111111111111111111111111111110000000000011101100110'

In [13]:
IV2_byte = base64.b64decode(IV2)
C2_byte =  base64.b64decode(C2)

AES = {} # AES table for the values we need
C_list = [IV2_byte] + [C2_byte[i:i+16] for i in range(0, len(C2_byte), 16)]

for C in C_list:
    N_int = int(N, 2)
    IV = BigEnd(N_int) + LitEnd(N_int)
    out = query_server_Q2(XOR_byte(IV, C))
    AES[C] = out
    query_server_Q2(M0); query_server_Q2(M0); query_server_Q2(M0)

    N = add(N, 8)

In [22]:
s = ""
for i in range(len(C_list) - 1):
    s += XOR_byte(AES[C_list[i]], C_list[i+1])
s

'Since we are encrypting this message with CBC mode, the length of the message must be a multiple of 16. That means that the length of random word clavier should be 7 modulo 16 because this message consists of 224 characters.'

In [15]:
# We don't know the most significant bit
# If the following ciphertext is not decrypted into meaningful ASCII, it means that the most significant bit is not the correct one. 
N = XOR(N, '1000000000000000000000000000000000000000000000000000000000000000')

# Return of Vernam cipher

In [33]:
p3=699802763607203174845032540020089692349581167953375281478177953237955229091099680081380600962387719273597
n3=699802763607203174845032540020089692349581167953375281478177953237955229091099680081380600962387719273598
k3=112965122454105818519362591698934638920439618839427449593542221146686839620173937856193008729230264260830
c3="..."

In [34]:
factor(n3)

2 * 3 * 2189713 * 2690659 * 2820017 * 2867959 * 3205519 * 3212147 * 3238231 * 3267727 * 3297683 * 3312077 * 3427747 * 3470003 * 3583357 * 3588469 * 3602749 * 3732719

In [35]:
factorList = []
for fac in (factor(n3)):
    factorList.append(fac[0])
    
allFactor = []
for i in range (1,3):
    for pairs in combinations(factorList,i):
        allFactor.append(numpy.prod(pairs))

In [36]:
cipher_bit = ''
K = []
cipher_bit = byte2bin(base2byte(c3))
cipherLength = len(cipher_bit)

In [37]:
len(cipher_bit)

187192

In [38]:
len(factorList)

18

In [39]:
for order in allFactor:
    order = int(order)
    XorCipher = ''
    if order in [2,3,6]:
        continue
    start = mod(k3,order)
    end = start+cipherLength-1
    mid = int(order/2)
    
    if (start<=mid and end <=mid) or (start>mid and end <=order-1):
        continue
    
    elif start <mid and end > mid:
        if end-mid>mid-start:
            L = mid - start
            left = cipher_bit[:L]
            right_reverse = cipher_bit[L+1:2*L+1]
            right = right_reverse[::-1]
            XorCipher = Xor(left,right,L)
            print('situation1')
        else:
            L = end - mid 
            right_reverse = cipher_bit[mid+1-start:end-start+1]
            left = cipher_bit[mid-L-start:mid-start]
            right = right_reverse[::-1]
            XorCipher = Xor(left,right,L)
            if XorCipher[:8]!='00000000':
                continue
            print('situation2')
                
    elif end < start:
        if end > order-1 - start :
            L = order-start
            left = cipher_bit[:order-start]
            right_reverse = cipher_bit[-start:L-start]
            right = right_reverse[::-1]
            XorCipher = Xor(left,right,L)
            print('situation3')
                
        elif end <=order - 1 - start :
            L = end
            right_reverse = cipher_bit[:end+1]
            left = cipher_bit[-start-L-1:-start]
            right = right_reverse[::-1]
            XorCipher = Xor(left,right,L)
            print('situation4')
            
    for i in range(0,len(XorCipher),8):
        if XorCipher[i:i+8]!='00000000':
            XorCipher = XorCipher[i:]
            break
    for j in range(len(XorCipher)-1,6,-8):
        if XorCipher[j-7:j+1]!='00000000':
            XorCipher = XorCipher[:j+1]
            break
    cut = len(XorCipher) % 8 
    XorCipher = XorCipher[:-cut]
    print(bin2byte(XorCipher))
    


situation1
It is always not easy to select a message with a random word such as repainting. Actually, I can just copy and paste some known dialgoues while replacing one word. But, I didn't do cause I don't want to find it


# ByteCipher

In [40]:
def encode_then_query_list(sciper, L):
    # L must be a list of tuples (m, iv0, ctext) as in encode_then_query
    ln = len(L)
    # ln can be max 100
    query = str(sciper) + " " + str(ln)
    for i in xrange(ln):
        (m, iv0, c) = L[i]
        query += " " + base64.b64encode(m) + " " + str(iv0) + " " + base64.b64encode(c)

    response = connect_server("lasecpc25.epfl.ch", int(5559), query)
    
    #print response
    try:
        response_list = response.split(" ")
        response_list = [int(x) for x in response_list]
        return (True, response_list)
    except:
        return (False, response)

In [41]:

# Use 
# base64.b64encode to convert non-printable strings into base64 strings
# base64.b64decode is the inverse


# Let's make a query

if len(sys.argv) < 2:
    print "I need your sciper as the first argument"
    exit(0)

sciper = sys.argv[1]

msg1 = "Hey, I want to ask you something....."
print len(msg1)
# let's make a ciphertext guess
IV1 = 3
c1 = "Maybe this is the ciphertext for msg!"
print len(c1)
IV2 = 217
c2 = "\x00___   __..awd\x98\x90\\//xx#\x98\x90\x00\x98r[.z]==\x90x8\x90"

sciper = 280678
# We can make a single query as below
L1 = [(msg1, IV1, c1)]
(valid , resp1) = encode_then_query_list(sciper, L1)
print((valid , resp1))
# ''valid'' tells me if my query was handled properly, i.e. the oracle returned a list
# filled with ones or zeros
# if ''valid'' is false, you can print the resp1 to see what went wrong

if not valid:
    print "Something went wrong with my query, here is the message"
    print resp1
    exit(1)
else:
    print "My query was executed just fine by the oracle (does not mean my pair was valid)"


if resp1 == [1]: # note that resp1 is a list of integers
    print "I must be too luck, this is a valid pair"
else:
    print "Luck is hard to find nowadays, my first pair is invalid."

# Let's make a list of queries simulateously to save time
L2 = [(msg1, IV1, c1), (msg1, IV2, c2)]
resp2 = encode_then_query_list(sciper, L2)

if resp2[1] == 1:
    print "My second ciphertext is valid, wow!"
else:
    print "No luck with the second"

print(resp2)




print len(c2)

37
37
(True, [0])
My query was executed just fine by the oracle (does not mean my pair was valid)
Luck is hard to find nowadays, my first pair is invalid.
No luck with the second
(True, [0, 0])
37


In [42]:
def hex2int(hex_num):
    int_num = int(hex_num,16) # use int base 16 to get the hex in int
    return int_num

def int2hex(int_num):
    hex_res = hex(int_num)
    return hex_res

def int2byte(int_num):
    hex_rep = int2hex(int_num)
    
    hex_rep = hex_rep.replace('0x','')
 
    if len(hex_rep)%2 != 0:
        hex_rep = '0' + hex_rep
    
    z = binascii.unhexlify(hex_rep)
    return z


In [43]:
Q4m = "kitchenette discouragements nonreturnables tonsillectomies readjustments segregationist vibraphones telemeters nationalistic commemorates assists egotistically electorate pharmacopeias moonstone commiserates mistrustful bureaucratic authoritarianism emphasizes."
Q4iv = 164

In [44]:
checker = ''
for i in range(len(Q4m)):
    send_list = []
    for ci in range(0,256):
        send_list.append((Q4m[:i+1],Q4iv,checker+int2byte(ci)))
        if ci % 64==63:
            z = encode_then_query_list(280678,send_list)
            if 1 in z[1]:
                print('we finish %d length',i)
                checker = send_list[z[1].index(1)][2]
                break
            else:
                send_list=[]

('we finish %d length', 0)
('we finish %d length', 1)
('we finish %d length', 2)
('we finish %d length', 3)
('we finish %d length', 4)
('we finish %d length', 5)
('we finish %d length', 6)
('we finish %d length', 7)
('we finish %d length', 8)
('we finish %d length', 9)
('we finish %d length', 10)
('we finish %d length', 11)
('we finish %d length', 12)
('we finish %d length', 13)
('we finish %d length', 14)
('we finish %d length', 15)
('we finish %d length', 16)
('we finish %d length', 17)
('we finish %d length', 18)
('we finish %d length', 19)
('we finish %d length', 20)
('we finish %d length', 21)
('we finish %d length', 22)
('we finish %d length', 23)
('we finish %d length', 24)
('we finish %d length', 25)
('we finish %d length', 26)
('we finish %d length', 27)
('we finish %d length', 28)
('we finish %d length', 29)
('we finish %d length', 30)
('we finish %d length', 31)
('we finish %d length', 32)
('we finish %d length', 33)
('we finish %d length', 34)
('we finish %d length', 35)
('

In [45]:
len(Q4m)

261

In [46]:
encode_then_query_list(280678,[[Q4m,Q4iv,checker]])

(True, [1])

In [47]:
byte2base(checker)

'z57k2yHawWBN22TayP1gwICBRsKIBpj51djGFGn1WwWw7l+arveOB5HoM93e7jmInMl+aupWyjG2simuglFvGc+itSV5GyMI7W9/h5NBLPlBtKkPR1MakEhuBk6M5iLuP9i5cqQOIWpOD2lI0Q/qhCX1ybzAatj18tyesrrUVHJYwKyOR2KRXe+MSP79B/9Jfplik1etN4+VY5Nd9+uWcMpPGhfd3lQXfBsHtx5ZSso3ysFUVRpRl6hJyJiL1yQnzC1N9N9EqU84T/zgPldTmO7/M/9Oaa0oNZ7yNE1FSepDFSzqv5k+URAllKWgu3tJ9HBrnsWAhe5y8cCsvZFiLMjssU1E'

In [48]:
len(byte2base(checker))

348

In [49]:
len(byte2base(Q4m))

348

In [50]:
cipher = ''
for s in checker:
    cipher +=base64.b64encode(s)

In [51]:
cipher

'zw==ng==5A==2w==IQ==2g==wQ==YA==TQ==2w==ZA==2g==yA==/Q==YA==wA==gA==gQ==Rg==wg==iA==Bg==mA==+Q==1Q==2A==xg==FA==aQ==9Q==Ww==BQ==sA==7g==Xw==mg==rg==9w==jg==Bw==kQ==6A==Mw==3Q==3g==7g==OQ==iA==nA==yQ==fg==ag==6g==Vg==yg==MQ==tg==sg==KQ==rg==gg==UQ==bw==GQ==zw==og==tQ==JQ==eQ==Gw==Iw==CA==7Q==bw==fw==hw==kw==QQ==LA==+Q==QQ==tA==qQ==Dw==Rw==Uw==Gg==kA==SA==bg==Bg==Tg==jA==5g==Ig==7g==Pw==2A==uQ==cg==pA==Dg==IQ==ag==Tg==Dw==aQ==SA==0Q==Dw==6g==hA==JQ==9Q==yQ==vA==wA==ag==2A==9Q==8g==3A==ng==sg==ug==1A==VA==cg==WA==wA==rA==jg==Rw==Yg==kQ==XQ==7w==jA==SA==/g==/Q==Bw==/w==SQ==fg==mQ==Yg==kw==Vw==rQ==Nw==jw==lQ==Yw==kw==XQ==9w==6w==lg==cA==yg==Tw==Gg==Fw==3Q==3g==VA==Fw==fA==Gw==Bw==tw==Hg==WQ==Sg==yg==Nw==yg==wQ==VA==VQ==Gg==UQ==lw==qA==SQ==yA==mA==iw==1w==JA==Jw==zA==LQ==TQ==9A==3w==RA==qQ==Tw==OA==Tw==/A==4A==Pg==Vw==Uw==mA==7g==/w==Mw==/w==Tg==aQ==rQ==KA==NQ==ng==8g==NA==TQ==RQ==SQ==6g==Qw==FQ==LA==6g==vw==mQ==Pg==UQ==EA==JQ==lA==pQ==oA==uw==ew==SQ==9A==cA==aw==ng==xQ==gA==hQ==7g==cg==8Q=

# Breaking the Feistel Network over Small Domain

In [52]:
pairs = read_data('./Asst3/281141/281141-parameters.txt',str)[20]

In [53]:
pairs = pairs.split('=')[1].replace('[','').replace(']','').split(',')
C5 = "YQU2sQkz"

In [54]:
def STRB(input_int,b):
    int_to_bin = bin(input_int)[2:]
    num_zeros = b - len(int_to_bin)
    int_to_bin = ('0'* num_zeros) + int_to_bin
    return int_to_bin

In [55]:
def NUM2(input_str):
    return int(input_str,2)

In [56]:
def hex2int(hex_num):
    int_num = int(hex_num,16) # use int base 16 to get the hex in int
    return int_num

In [57]:
def int2hex(int_num):
    hex_res = hex(int_num)[2:]
    return hex_res

In [58]:
def str2int(str_input):
    return hex2int(binascii.hexlify(base64.b64decode(str_input)))

In [59]:
def int2str(int_input):
    return base64.b64encode(binascii.unhexlify(int2hex(int_input)))

In [60]:
def process(bits,i,sciper):
    eight = STRB(i,8)
    sciper_bit = STRB(sciper,20)
    bits_bits = STRB(bits,100)
    final_conc = eight + sciper_bit + bits_bits
    return final_conc

In [61]:
pairs = map(lambda x: x.replace("'",'').replace('(','').replace(')',''),pairs)

In [62]:
def expand_or_reduce(bits):
    num_zeros = 48 - len(bits)
    new_bits = '0'*num_zeros + bits
    return new_bits

In [63]:
C5_bit = str2int(C5)
C5_bit = '0' + bin(C5_bit)[2:]
C5_z   = NUM2(C5_bit [0:24]) 
C5_t   = NUM2(C5_bit [24:])

In [64]:
def processText(text):
    text_bits = expand_or_reduce(bin(str2int(text))[2:])
    left = NUM2(text_bits[:24])
    right =NUM2(text_bits[24:])
    return left,right

In [65]:
X= []
Z= []
Y= []
T= []

for i in range(0,len(pairs),2):
    randx,randy = processText(pairs[i])
    randz,randt = processText(pairs[i+1])
    if randt  not in T:
        T.append(randt)
    else:
        print i

70
72
116
132
138
140
148
150
152
154
170
182
190
192
202
206
214
234
240
244
250
252
256
264
266
270
276
282
286
288
292
300
302
308
314
316
328
334
342
350
352
366
368
374
378
384
386
390
398
402
406
408
418
420
422
430
432
434
440
442
448
450
452
456
458
460
462
464
468
470
474
476
478
480
482
486
488
490
492
496
500
502
508
510
512
516
520
522
524
528
530
540
542
544
546
548
556
558
560
562
568
570
574
576
578
582
584
586
588
590
592
594
598
602
604
606
608
610
612
614
616
618
620
622
626
628
634
636
638
640
642
644
646
650
652
654
658
660
662
666
668
670
674
676
680
682
684
686
688
690
692
694
696
698
700
702
704
706
708
710
712
714
716
718
722
724
726
728
730
732
734
736
738
740
742
744
746
748
752
754
758
760
762
764
766
768
770
772
774
776
778
780
782
786
788
790
792
794
796
798


In [66]:
f0,f1,f2 = dict(),dict(),dict()
N = 2**24

# randx,randy = processText(pairs[70])
# randz,randt = processText(pairs[71])

randx,randy = processText(pairs[70]) # a t repeated twice
randz,randt = processText(pairs[71])
# add them to dictionary
f0[randy] = 0
f2[randt] = mod(randz - randx,N)
f1[randx] = mod(randt - randy,N)
index_list = set()
index_list.add(70)
k = 0
while True:
    
    for i in range(0,len(pairs),2):
    
        if i not in index_list:
            plain_text_x,plain_text_y   = processText(pairs[i])
            cipher_text_z,cipher_text_t = processText(pairs[i+1])
            if (cipher_text_t in f2.keys()):
                a = 2
            else: 
                a = 0
            if (plain_text_y in f0.keys()):
                b = 1
            else: 
                b = 0   
            
            if (a+b)==0:
                continue
            elif (a+b)==1: 
                
                c = mod(f0[plain_text_y] + plain_text_x,N)
                f1[c] = mod(cipher_text_t - plain_text_y,N)
                f2[cipher_text_t] = mod(cipher_text_z - c,N)
                
            elif(a+b)==2:
                c = mod(cipher_text_z - f2[cipher_text_t],N)
                f0[plain_text_y] = mod(c-plain_text_x,N)
                f1[c] = mod(cipher_text_t - plain_text_y,N)
            else:
                continue
            
            index_list.add(i)
    k+=1
    if C5_t in f2.keys():
        c_5 = mod(C5_z - f2[C5_t],N)
        if c_5 in f1.keys():
            c5_y = mod(C5_t - f1[c_5],N)
            if c5_y in f0.keys():
                c5_x = mod(c_5 - f0[c5_y],N)
                print("Decryption X = {} and Y = {}".format(c5_x,c5_y))
                break

Decryption X = 5097617 and Y = 3062386


In [67]:
X_bin,Y_bin = bin(5097617)[2:],bin(3062386)[2:]

In [68]:
result = X_bin + Y_bin

In [69]:
len(result)

45

In [70]:
byte2base(bin2byte('000' + result))

'E3Ikbrpy'